In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TERRAIN
from ast import literal_eval
from scipy import stats
from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b_20
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot

In [2]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
airbnb_cr = pd.read_csv('/home/novais/Downloads/airbnb_cr.csv')

In [4]:
listings = pd.read_csv('/home/novais/Downloads/listings.csv')

In [5]:
crimes = pd.read_csv('/home/novais/Downloads/crimes.csv')

In [6]:
crimes = crimes[crimes.lat>40]
crimes = crimes[crimes.desc.isin(["BURGLARY", "THEFT", "ROBBERY", "MOTOR VEHICLE THEFT"])]
crimes.head()

,Unnamed: 0,date,latitude,longitude,arrest,desc,lat,lon,location,loc,loc_id,month,day,year
5,466,2018-07-09,41.937721,-87.640515,1,MOTOR VEHICLE THEFT,41.94,-87.64,"(41.937720768, -87.640514509)","(41.94, -87.64)",6351,7,9,2018
9,471,2018-07-09,41.857944,-87.709081,1,THEFT,41.86,-87.71,"(41.857943829, -87.709080933)","(41.86, -87.71)",6201,7,9,2018
10,472,2018-07-09,41.859747,-87.630271,1,THEFT,41.86,-87.63,"(41.859746945, -87.63027064299999)","(41.86, -87.63)",6209,7,9,2018
12,474,2018-07-09,41.692780,-87.600611,0,THEFT,41.69,-87.60,"(41.692780068000005, -87.600611049)","(41.69, -87.6)",5930,7,9,2018
13,475,2018-07-09,41.909271,-87.647229,0,THEFT,41.91,-87.65,"(41.909270561, -87.64722927)","(41.91, -87.65)",6292,7,9,2018


In [7]:
cr_count=crimes.groupby(['loc_id']).count()
cr_count=cr_count.desc.reset_index()
cr_count=cr_count.rename(index=str, columns={"desc": "crimes_count"})
cr_count.head()

,loc_id,crimes_count
0,0,2413
1,4808,2
2,4884,1
3,5001,19
4,5021,4


In [8]:
crimes2=crimes.merge(cr_count,on='loc_id')
crimes2=crimes2.groupby('loc_id').mean()
crimes2=crimes2.reset_index()
crimes2=crimes2[['lat','lon','loc_id','crimes_count']]
crimes2['loc']=list(zip(crimes2.lat,crimes2.lon))
crimes2.head()

,lat,lon,loc_id,crimes_count,loc
0,41.89,-87.63,0,2413.0,"(41.889999999999475, -87.63000000000287)"
1,41.73,-87.72,4808,2.0,"(41.73, -87.72)"
2,41.68,-87.56,4884,1.0,"(41.68, -87.56)"
3,41.68,-87.60,5001,19.0,"(41.679999999999986, -87.59999999999997)"
4,41.67,-87.54,5021,4.0,"(41.67, -87.54)"


In [9]:
crimes2['new_count'] = crimes2['crimes_count'].apply([lambda x: 10 if x<58 else(10 if x<500 else 20) ])

In [10]:
crimes2.head()

,lat,lon,loc_id,crimes_count,loc,new_count
0,41.89,-87.63,0,2413.0,"(41.889999999999475, -87.63000000000287)",20
1,41.73,-87.72,4808,2.0,"(41.73, -87.72)",10
2,41.68,-87.56,4884,1.0,"(41.68, -87.56)",10
3,41.68,-87.60,5001,19.0,"(41.679999999999986, -87.59999999999997)",10
4,41.67,-87.54,5021,4.0,"(41.67, -87.54)",10


In [11]:
crimes2.crimes_count.describe()

count     681.000000
mean      130.355360
std       204.173002
min         1.000000
25%        27.000000
50%        89.000000
75%       173.000000
max      3352.000000
Name: crimes_count, dtype: float64

In [12]:
# crimes2['location'] = crimes2['location'].apply(lambda x: literal_eval(x))
crimes2['coords_x'] = crimes2['loc'].apply(lambda x: merc(x)[0])
crimes2['coords_y'] = crimes2['loc'].apply(lambda x: merc(x)[1])

c1 = crimes2[crimes2['crimes_count']<25]
c2 = crimes2[(crimes2['crimes_count']>=25) & (crimes2['crimes_count']<=100)]
c3 = crimes2[(crimes2['crimes_count']>100) & (crimes2['crimes_count']<=200)]
c4 = crimes2[(crimes2['crimes_count']>200) & (crimes2['crimes_count']<=500)]
c5 = crimes2[crimes2['crimes_count']>500]

p1 = figure(title = "Crimes Count")
p1.xaxis.axis_label = 'Latitude'
p1.yaxis.axis_label = 'Longitude'

source1 = ColumnDataSource(data=dict(
    count=c1['crimes_count'],
    cx=c1['coords_x'],
    cy=c1['coords_y'],
    names = ['< 25']*len(c1),
    colors = ['#FFB401']*len(c1),
    sizes = [10]*len(c1)))
source2 = ColumnDataSource(data=dict(
    count=c2['crimes_count'],
    cx=c2['coords_x'],
    cy=c2['coords_y'],
    names = ['[25,100]']*len(c2),
    colors = ['#F39A05']*len(c2),
    sizes = [10]*len(c2)))
source3 = ColumnDataSource(data=dict(
    count=c3['crimes_count'],
    cx=c3['coords_x'],
    cy=c3['coords_y'],
    names = ['(100,200]']*len(c3),
    colors = ['#E57505']*len(c3),
    sizes = [10]*len(c3)))
source4 = ColumnDataSource(data=dict(
    count=c4['crimes_count'],
    cx=c4['coords_x'],
    cy=c4['coords_y'],
    names = ['(200,500]']*len(c4),
    colors = ['#D34B08']*len(c4),
    sizes = [10]*len(c4)))
source5 = ColumnDataSource(data=dict(
    count=c5['crimes_count'],
    cx=c5['coords_x'],
    cy=c5['coords_y'],
    names = ['> 500']*len(c5),
    colors = ['#C2240B']*len(c5),
    sizes = [10]*len(c5)))


p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'cx',y = 'cy',source=source1,color='colors',legend='names',size='sizes')
p1.circle(x = 'cx',y = 'cy',source=source2,color='colors',legend='names',size='sizes')
p1.circle(x = 'cx',y = 'cy',source=source3,color='colors',legend='names',size='sizes')
p1.circle(x = 'cx',y = 'cy',source=source4,color='colors',legend='names',size='sizes')
p1.circle(x = 'cx',y = 'cy',source=source5,color='colors',legend='names',size='sizes')

p1.add_tools(HoverTool(
    tooltips=[
        ( 'Nº of occurrences', '@count')
    ],
    
    formatters={
        'Nº of occurrences' : 'numeral',
    },

))

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p1_script, p1_div = components(p1)

In [13]:
output_notebook()


Loading BokehJS ...

In [15]:
show(p1)

In [16]:
print(p1_script)


<script type="text/javascript">
  (function() {
    var fn = function() {
      Bokeh.safely(function() {
        (function(root) {
          function embed_document(root) {
            
          var docs_json = '{"1afcedfe-f919-494b-b095-226a0a9c363d":{"roots":{"references":[{"attributes":{},"id":"1115","type":"UnionRenderers"},{"attributes":{"label":{"field":"names"},"renderers":[{"id":"1105","type":"GlyphRenderer"}]},"id":"1116","type":"LegendItem"},{"attributes":{"callback":null,"data":{"colors":["#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B","#C2240B"],"count":{"__ndarray__":"AAAAAADaokAAAAAAADiBQAAAAAAAGIBAAAAAAACQhkAAAAAAALiCQAAAAAAAMKpAAAAAAADYhEAAAAAAAKB/QAAAAAAA0JNAAAAAAACwf0AAAAAAAOiSQAAAAAAACIpAAAAAAACohEAAAAAAAMCAQAAAAAAAuIZAAAAAAAAQgUAAAAAAAGiJQA==","dtype":"float64","shape":[17]},"cx":{"__ndarray__":"TIlN3yWbYsF8iE3fJZtiwRY4igWxm2LBYIhN3yWbYsEVOIoFsZtiwWmJTd

In [17]:
print(p1_div)


<div class="bk-root" id="983be363-469e-484f-82e7-02925ca8938b"></div>
